In [1]:
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from model import RegressionModel
import numpy as np
from numpy import load
import os
import sys

In [ ]:
#model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

hidden_sizes = [512, 256, 128, 64]
model = RegressionModel(input_size=5, hidden_sizes=hidden_sizes, output_size=1, num_layers=5, dropout=0.1)
model

In [3]:
#evaluate patient by patient

def evaluate(organ):

    #model loading for the organ
    model.load_state_dict(torch.load('chest_models/'+ organ + '_model.pth'))
    model.to(device)

    #scaler for the organ
    data = load('training_arrays/chest_merged_training_arrays/'+ organ + '_training_array.npy')

    X = data[:,:-1]

    # Find rows containing NaN values in X
    nan_rows_X = np.isnan(X).any(axis=1)

    # Drop rows with NaN values from X 
    X = X[~nan_rows_X]

    # fit scaler 
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    # Save the original standard output
    original_stdout = sys.stdout

    # Define the file name for saving the output
    output_filename ='validation_results/' + organ + '_validation_results.txt'

    # Open the file for writing
    with open(output_filename, 'w') as f:
        # Redirect the standard output to the file
        sys.stdout = f

        files = os.listdir('chest_validation_arrays/' + organ)
        diffs = []
        pred_dose = []
        gt_dose = []
        
        for file in files:
            
            print(file)
            
            test_data = load('chest_validation_arrays/' + organ + '/' + file)
            X_test = test_data[:,:-1]
            y_test = test_data[:,-1]
            
            # Find rows containing NaN values in X
            nan_rows_X_test = np.isnan(X_test).any(axis=1)

            # Drop rows with NaN values from X and y
            X_test = X_test[~nan_rows_X_test]
            y_test = y_test[~nan_rows_X_test]
            
            #scaling
            X_test = scaler.transform(X_test)

            #data to tensor
            X_test = torch.tensor(X_test, dtype=torch.float32).to(device)

            #evaluate
            with torch.no_grad():
                model.eval()
                predictions = model(X_test).cpu().numpy()

            diffs.append(abs(100*((np.mean(y_test) - np.mean(predictions))/np.mean(y_test))))
            pred_dose.append(np.mean(predictions))
            gt_dose.append(np.mean(y_test))

            print(f'Ground Truth dose (mGy): {np.mean(y_test):.1f}')
            print(f'Predicted dose (mGy): {np.mean(predictions):.1f}')
            print(f'Percent Error:  {100*((np.mean(y_test) - np.mean(predictions))/np.mean(y_test)):.1f}%')
            print('----------------------')
        print('Results')
        print('----------------------')
        print(f'Ground Truth dose (mGy): {np.mean(gt_dose):.1f}')
        print(f'Standard deviation (mGy): {np.std(gt_dose):.1f}')
        print('----------------------')
        print(f'Predicted dose (mGy): {np.mean(pred_dose):.1f}')
        print(f'Standard deviation (mGy): {np.std(pred_dose):.1f}')
        print('----------------------')
        print(f'Median Percentage difference: {np.nanmedian(diffs):.1f}%')
        print(f'Q1: {np.nanpercentile(diffs, 25):.1f}%')
        print(f'Q3: {np.nanpercentile(diffs, 75):.1f}%')
        print(f'IQR: {(np.nanpercentile(diffs, 75) - np.nanpercentile(diffs, 25)):.1f}%')
        print('----------------------')
        print(f'Mean Percentage difference: {np.nanmean(diffs):.1f}%')
        print(f'Max Percentage difference: {np.nanmax(diffs):.1f}%')
        print(f'Min Percentage difference: {np.nanmin(diffs):.1f}%')
        print(f'Standard deviation: {np.nanstd(diffs):.1f}%')
        sys.stdout = original_stdout


In [4]:
#Validation for chest organs

organs_list = ['esophagus', 'aorta', 'pulmonary_artery', 'lung_lower_lobe_left', 'lung_lower_lobe_right',
                    'lung_upper_lobe_left', 'lung_upper_lobe_right', 'lung_middle_lobe_right', 
                    'heart_atrium_left', 'heart_atrium_right', 'heart_ventricle_left', 'heart_ventricle_right', 
                    'heart_myocardium', 'skin', 'lung', 'lung_vessels', 'lung_trachea_bronchia']

for organ in organs_list:
    evaluate(organ)